In [ ]:
## Drivera bağlanmak için
from google.colab import drive
drive.mount('/content/drive')
vid='018'

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/TEZ2')

In [ ]:
#inb_mass_seg_01 seti kullanınca alttaki kodu çalıştır.
#from Masssegmentationmain.modelUnet_nikhil import *
#inb_mass_seg_02 seti kullanınca alttaki kodu çalıştır üstekini kapat
from Masssegmentationmain.modelUnet_nikhill import *
#inb_mass_seg_01 seti kullanınca alttaki kodu çalıştır
#from Masssegmentationmain.data_1ch import *
#inb_mass_seg_02 seti kullanınca alttaki kodu çalıştır üstekini kapat
from Masssegmentationmain.data_2ch import *
#from Mass-segmentation-main.data_1ch import *
import os, os.path



# Count the number of train and valid files
train_dir = '/content/drive/MyDrive/inb_mass_seg_02/train/mg'
train_count=len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])

valid_dir = '/content/drive/MyDrive/inb_mass_seg_02/valid/mg'
valid_count=len([name for name in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, name))])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=90,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=2
learning_rate=1e-4

train_gen = trainGenerator(batch_size,'/content/drive/MyDrive/inb_mass_seg_02/train','mg','mask',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'/content/drive/MyDrive/inb_mass_seg_02/valid','mg','mask',data_gen_args,save_to_dir = None)

# train_count images are used for training, valid_count images for validating
train_steps = train_count//batch_size
valid_steps = valid_count//batch_size


# SETTINGS ***
loss=dice_loss
steps_per_epoch=3*train_steps
num_epochs=100
num_top_filter=32

#model = unet(learning_rate, loss, num_top_filter)
model = unet(num_top_filter)

opt = tf.keras.optimizers.Adam(learning_rate)
#metrics = ["acc", dice_coef, iou]
metrics = ["acc", dice_coef]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('/content/drive/MyDrive/TEZ2/bestmodel/chkpt'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("/content/drive/MyDrive/TEZ2/bestmodel/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps,
                    epochs=num_epochs, callbacks=callbacks)

Found 68 images belonging to 1 classes.
Found 68 images belonging to 1 classes.


<ipython-input-4-71e1578f1411>:46: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps,


Epoch 1/100
102/102 [==============================] - ETA: 0s - loss: 0.9239 - acc: 0.8857 - dice_coef: 0.0761Found 17 images belonging to 1 classes.
Found 17 images belonging to 1 classes.

Epoch 1: saving model to /content/drive/MyDrive/TEZ2/bestmodel/chkpt018.hdf5
102/102 [==============================] - 163s 1s/step - loss: 0.9239 - acc: 0.8857 - dice_coef: 0.0761 - val_loss: 0.9657 - val_acc: 0.9817 - val_dice_coef: 0.0343 - lr: 1.0000e-04
Epoch 2/100
102/102 [==============================] - ETA: 0s - loss: 0.8919 - acc: 0.9375 - dice_coef: 0.1081
Epoch 2: saving model to /content/drive/MyDrive/TEZ2/bestmodel/chkpt018.hdf5
102/102 [==============================] - 29s 284ms/step - loss: 0.8919 - acc: 0.9375 - dice_coef: 0.1081 - val_loss: 0.9621 - val_acc: 0.0809 - val_dice_coef: 0.0379 - lr: 1.0000e-04
Epoch 3/100
102/102 [==============================] - ETA: 0s - loss: 0.8708 - acc: 0.9492 - dice_coef: 0.1292
Epoch 3: saving model to /content/drive/MyDrive/TEZ2/bestmodel

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
learning_rate=learning_rate
loss=loss
num_top_filter=num_top_filter
model = unet(num_top_filter)

In [ ]:
validGene = testGenerator("/content/drive/MyDrive/inb_mass_seg_02/valid/pred")
model.load_weights('/content/drive/MyDrive/TEZ2/bestmodel/chkpt'+vid+'.hdf5')
results = model.predict_generator(validGene,valid_count,verbose=1)
saveResult('/content/drive/MyDrive/inb_mass_seg_02/valid/pred',results,vid[:4])

<ipython-input-6-20ee554dc4e3>:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(validGene,valid_count,verbose=1)


17/17 [==============================] - 13s 662ms/step


/content/drive/MyDrive/TEZ2/Masssegmentationmain/data_2ch.py:104: UserWarning: /content/drive/MyDrive/inb_mass_seg_02/valid/pred/0_predict_018.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
/content/drive/MyDrive/TEZ2/Masssegmentationmain/data_2ch.py:104: UserWarning: /content/drive/MyDrive/inb_mass_seg_02/valid/pred/2_predict_018.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
/content/drive/MyDrive/TEZ2/Masssegmentationmain/data_2ch.py:104: UserWarning: /content/drive/MyDrive/inb_mass_seg_02/valid/pred/6_predict_018.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
/content/drive/MyDrive/TEZ2/Masssegmentationmain/data_2ch.py:104: UserWarning: /content/drive/MyDrive/inb_mass_seg_02/valid/pred/9_predict_018.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
/content/drive/MyDrive/TEZ2/Masssegmenta